In [13]:
import duckdb
import pandas as pd

db_path = "/Users/folkert/think_onward/data/eaglei_data.duckdb"

with duckdb.connect(database=db_path) as conn:
    df = conn.execute("SELECT * FROM eaglei_data WHERE customers_out IS NOT NULL LIMIT 10;").fetchdf()
    display(df)

,fips_code,county,state,customers_out,run_start_time
0,01037,Coosa,Alabama,12,2014-11-01 04:00:00
1,01051,Elmore,Alabama,7,2014-11-01 04:00:00
2,01109,Pike,Alabama,1,2014-11-01 04:00:00
3,01121,Talladega,Alabama,31,2014-11-01 04:00:00
4,04017,Navajo,Arizona,1,2014-11-01 04:00:00
5,05009,Boone,Arkansas,3,2014-11-01 04:00:00
6,05119,Pulaski,Arkansas,1,2014-11-01 04:00:00
7,06029,Kern,California,30,2014-11-01 04:00:00
8,06037,Los Angeles,California,1555,2014-11-01 04:00:00
9,06065,Riverside,California,2,2014-11-01 04:00:00


In [ ]:
import pandas as pd
import duckdb

def process_state_county(conn, state, county):
    if "'" in county:
        county = county.replace("'", "''")
    query = f"""
    SELECT
        state,
        county,
        customers_out,
        run_start_time
    FROM
        eaglei_data
    WHERE
        customers_out IS NOT NULL
        AND customers_out > 0
        AND state = '{state}'
        AND county = '{county}'
    ORDER BY run_start_time
    """

    df = conn.sql(query).df()
    df['group'] = (df['run_start_time'] != df['run_start_time'].shift() + pd.Timedelta('15 minutes')).cumsum()

    summary = df.groupby('group').agg({
        'state': 'first',
        'county': 'first',
        'run_start_time': ['first', 'last'],
        'customers_out': ['min', 'max', 'mean']
    }).reset_index()

    summary.columns = ['group', 'state', 'county', 'start_time', 'end_time', 'min_customers', 'max_customers', 'avg_customers']
    summary['duration'] = summary['end_time'] - summary['start_time']

    return summary

def get_state_county_pairs(conn):
    query = """
    SELECT DISTINCT state, county
    FROM eaglei_data
    WHERE customers_out IS NOT NULL
    ORDER BY state, county
    """
    return conn.sql(query).df()

def process_all_data(db_path):
    results = []

    with duckdb.connect(database=db_path) as conn:
        pairs = get_state_county_pairs(conn)
        total_pairs = len(pairs)

        for idx, (_, row) in enumerate(pairs.iterrows(), 1):
            print(f"Processing {row['state']}, {row['county']} ({idx}/{total_pairs})")
            summary = process_state_county(conn, row['state'], row['county'])
            results.append(summary)

    final_summary = pd.concat(results, ignore_index=True)
    final_summary['group'] = range(len(final_summary))

    return final_summary

outage_summary = process_all_data(db_path)

Processing Alabama, Autauga (1/3095)
Processing Alabama, Baldwin (2/3095)
Processing Alabama, Barbour (3/3095)
Processing Alabama, Bibb (4/3095)
Processing Alabama, Blount (5/3095)
Processing Alabama, Bullock (6/3095)
Processing Alabama, Butler (7/3095)
Processing Alabama, Calhoun (8/3095)
Processing Alabama, Chambers (9/3095)
Processing Alabama, Cherokee (10/3095)
Processing Alabama, Chilton (11/3095)
Processing Alabama, Choctaw (12/3095)
Processing Alabama, Clarke (13/3095)
Processing Alabama, Clay (14/3095)
Processing Alabama, Cleburne (15/3095)
Processing Alabama, Coffee (16/3095)
Processing Alabama, Colbert (17/3095)
Processing Alabama, Conecuh (18/3095)
Processing Alabama, Coosa (19/3095)
Processing Alabama, Covington (20/3095)
Processing Alabama, Crenshaw (21/3095)
Processing Alabama, Cullman (22/3095)
Processing Alabama, Dale (23/3095)
Processing Alabama, Dallas (24/3095)
Processing Alabama, DeKalb (25/3095)
Processing Alabama, Elmore (26/3095)
Processing Alabama, Escambia (27/

In [53]:
print(outage_summary.sort_values('duration', ascending=False).head(10).to_markdown())

|          |    group | state         | county       | start_time          | end_time            |   min_customers |   max_customers |   avg_customers | duration          |
|---------:|---------:|:--------------|:-------------|:--------------------|:--------------------|----------------:|----------------:|----------------:|:------------------|
|  5447407 |  5447407 | Massachusetts | Hampden      | 2022-01-01 00:00:00 | 2022-08-23 18:30:00 |               1 |           23142 |       215.161   | 234 days 18:30:00 |
|   779627 |   779627 | California    | Los Angeles  | 2022-05-31 13:30:00 | 2022-10-18 23:00:00 |               1 |           93099 |      2027.89    | 140 days 09:30:00 |
|   779573 |   779573 | California    | Los Angeles  | 2020-07-12 11:15:00 | 2020-10-20 10:45:00 |               1 |           65931 |      2349.69    | 99 days 23:30:00  |
|  7265819 |  7265819 | New Jersey    | Morris       | 2022-05-15 11:45:00 | 2022-08-19 17:15:00 |               4 |           27353 | 

In [23]:
display(outage_summary.sort_values('avg_customers', ascending=False).head(10))

,group,state,county,start_time,end_time,min_customers,max_customers,avg_customers,duration
1570579,1570579,Florida,Miami-Dade,2017-09-11 13:45:00,2017-09-11 14:15:00,808450,1618030,1.078687e+06,0 days 00:30:00
1570578,1570578,Florida,Miami-Dade,2017-09-11 01:30:00,2017-09-11 13:15:00,775220,1777800,8.630394e+05,0 days 11:45:00
1570580,1570580,Florida,Miami-Dade,2017-09-11 14:45:00,2017-09-11 18:30:00,802430,1616900,8.629662e+05,0 days 03:45:00
1276503,1276503,Florida,Broward,2017-09-11 13:45:00,2017-09-11 14:15:00,636850,1275330,8.502200e+05,0 days 00:30:00
1627889,1627889,Florida,Palm Beach,2017-09-11 13:45:00,2017-09-11 14:15:00,517690,1044000,6.960000e+05,0 days 00:30:00
1276502,1276502,Florida,Broward,2017-09-11 01:30:00,2017-09-11 13:15:00,632320,1398920,6.765910e+05,0 days 11:45:00
1276504,1276504,Florida,Broward,2017-09-11 14:45:00,2017-09-11 18:30:00,629080,1273700,6.741394e+05,0 days 03:45:00
1627890,1627890,Florida,Palm Beach,2017-09-11 14:45:00,2017-09-11 18:30:00,509490,1035380,5.530706e+05,0 days 03:45:00
1627888,1627888,Florida,Palm Beach,2017-09-11 01:30:00,2017-09-11 13:15:00,523140,1098160,5.491340e+05,0 days 11:45:00
1570581,1570581,Florida,Miami-Dade,2017-09-11 19:00:00,2017-09-14 11:45:00,314040,1604860,5.466466e+05,2 days 16:45:00


In [29]:
len(outage_summary[outage_summary['min_customers'] > 100])

327841

In [ ]:
state = 'Virginia'
county = 'Albemarle'

query = f"""
    SELECT
        state,
        county,
        customers_out,
        run_start_time
    FROM
        eaglei_data
    WHERE
        customers_out IS NOT NULL
        AND customers_out > 0
        AND state = '{state}'
        AND county = '{county}'
    ORDER BY run_start_time
    """

with duckdb.connect(database=db_path) as conn:
    df = conn.execute(query).fetchdf()
    display(df[(df['run_start_time'] >= '2021-12-28') & (df['run_start_time'] <= '2022-01-01')])

,state,county,customers_out,run_start_time
83085,Massachusetts,Hampden,10,2021-12-28 00:00:00
83086,Massachusetts,Hampden,10,2021-12-28 00:15:00
83087,Massachusetts,Hampden,10,2021-12-28 00:30:00
83088,Massachusetts,Hampden,10,2021-12-28 00:45:00
83089,Massachusetts,Hampden,10,2021-12-28 01:00:00
...,...,...,...,...
83370,Massachusetts,Hampden,16,2021-12-30 23:15:00
83371,Massachusetts,Hampden,16,2021-12-30 23:30:00
83372,Massachusetts,Hampden,16,2021-12-30 23:45:00
83373,Massachusetts,Hampden,16,2021-12-31 00:00:00
